# 特異クラス

* 特異クラス
    * 指定したインスタンスだけに適用される特別なクラス
        * 無名クラス(モジュールがインクルードされたときにそのメソッドが属するクラス)と同様にインタプリタ内部で使用するクラス
            * ユーザに意識させたくないクラス
        * 無名クラスと混同しないように気を付ける
    * Silver試験の出題範囲外

## 特異クラスの性質

* 特定のインスタンスに特別な性質を持たせたい
    * 継承したクラスを書いて新たにインスタンス化するほどではない
    * いちいちサブクラスを定義するのは面倒くさい
        * このときは特異メソッドを使う
* 特異メソッド
    * インスタンスに直接定義したメソッド

In [13]:
class Foo
end
foo1 = Foo.new
foo2 = Foo.new

### インスタンスに直接メソッドを定義 (特異メソッドの定義)
def foo1.methodA
  1 + 1
end
puts foo1.methodA
puts "foo1のクラスはFooのまま"
puts foo1.class

puts "foo2にmethodAは存在しない"
puts foo2.methodA


2
foo1のクラスはFooのまま
Foo
foo2にmethodAは存在しない


NoMethodError: undefined method `methodA' for #<Foo:0x007f083f08df88>

* 仕組み
    1. 特異メソッドを定義する
    1. Rubyインタプリタは特異クラスと呼ばれるクラスを作成した継承チェーンに組み込む
        * foo1の特異クラスは「#foo1」と表現する
        * 特異クラス#foo1のsuperclassはFooクラス (#foo < Foo のようなイメージ)
        * foo1のクラスメソッドはFooクラスを指しているまま

## 特異クラスの参照

* 特異クラスは通常ユーザに意識させたくないものであるが、参照は可能

特異クラスの再オープン式
``` ruby
class << (対象オブジェクト)
end
```

In [25]:
class Foo
end
foo1 = Foo.new

SingletonClassA = class << foo1
  self
end

puts SingletonClassA
puts SingletonClassA.superclass

#<Class:#<Foo:0x007f083f15a088>>
Foo


## 特異クラスの再オープンとメソッドの定義

` def <オブジェクト名>.<新たに定義するメソッド名> ... end ` でなくても特異クラスの再オープン式でメソッドを定義できる

In [28]:
class Foo
end
foo1 = Foo.new
foo2 = Foo.new

### defを使った特異メソッドの定義
def foo1.methodA
  1 + 1
end

### 特異クラスの再オープンと特異メソッド定義
class << foo2
  def methodB
    2 + 2
  end
end

puts foo1.methodA
puts foo2.methodB

2
4


## selfとメソッドの定義先

* self
    * 自分自身を示す特別なオブジェクト
    * どこで使われるかによって示すオブジェクトが異なる
        * クラス内部
            * そのクラスを保持
        * メソッド内部
            * そのメソッドを実行するオブジェクトを保持
* レシーバ
    * メソッドが実行されるオブジェクトのこと
    * 「メソッドの受け手」という意味

In [47]:
class C1
  self
  def methodD
    self
  end
end

c1 = C1.new
puts "クラス内部で呼び出したself"
puts c1
puts "クラスの中のインスタンスメソッド内部で呼び出したself(レシーバ)"
puts c1.methodD
puts "クラスのオブジェクトとレシーバは同じオブジェクト(そのクラスを参照している)"
puts c1 == c1.methodD

クラス内部で呼び出したself
#<C1:0x007f083e836588>
クラスの中のインスタンスメソッド内部で呼び出したself(レシーバ)
#<C1:0x007f083e836588>
クラスのオブジェクトとレシーバは同じオブジェクト(そのクラスを参照している)
true


## メソッドのネスト

* メソッドは定義したクラスに所属する
* メソッドがネストした場合、内側のメソッドは外側のメソッドが定義されたクラスに定義される
    * ただし、<font color="red">外側のメソッドが実行されるまでは定義されない</font>


In [46]:
class C3
  def methodE
    def methodF
    end
  end
end

puts "まだmethodFは定義されていない"
puts C3.instance_methods(false)
C3.new.methodE
puts "methodEが実行されたためmethodFが定義された"
puts C3.instance_methods(false)


まだmethodFは定義されていない
[:methodE]
methodEが実行されたためmethodFが定義された
[:methodE, :methodF]


## 特異クラスに対するMix-in

* include
    * クラスにモジュールをincludeする
* extend
    * 特異クラスへのMix-inで良く使用する
    * includeよりシンプルに書ける
* prepend
    * includeと同様にモジュールのメソッドをクラスに取り込むためのメソッド
    * クラスよりも先にモジュールのメソッド探索を行う



In [67]:
module Bar
  def methodA
    3+4
  end
end

class Foo
end

foo1 = Foo.new
foo2 = Foo.new

### includeを使ったMix-in
class << foo1
  include Bar
end
puts foo1.methodA

### extendメソッドを使ったMix-in
### foo2インスタンスでモジュールBarのメソッドmethodAを実行
puts foo2.extend(Bar).methodA

7
7
